In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dataworkshop-project/Road signs recognition'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dataworkshop-project/Road signs recognition


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  
  return accuracy_score(y_test_norm, y_pred)

In [0]:

def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
  ])

In [0]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 17s 63ms/step - loss: 2.4641 - accuracy: 0.2956 - val_loss: 1.2220 - val_accuracy: 0.6228
Epoch 2/5
272/272 [==============================] - 16s 61ms/step - loss: 0.9765 - accuracy: 0.6860 - val_loss: 0.3191 - val_accuracy: 0.9056
Epoch 3/5
272/272 [==============================] - 16s 61ms/step - loss: 0.4607 - accuracy: 0.8566 - val_loss: 0.1275 - val_accuracy: 0.9634
Epoch 4/5
272/272 [==============================] - 16s 60ms/step - loss: 0.2932 - accuracy: 0.9103 - val_loss: 0.0862 - val_accuracy: 0.9741
Epoch 5/5
272/272 [==============================] - 16s 60ms/step - loss: 0.2160 - accuracy: 0.9343 - val_loss: 0.0646 - val_accuracy: 0.9817


0.9578231292517007

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),
    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 120.0, 'dropout_cnn_block_one': 0.4915487561336791, 'dropout_cnn_block_three': 0.30724383375394515, 'dropout_cnn_block_two': 0.40885286394143744, 'dropout_dense_block_one': 0.6455251610555532, 'dropout_dense_block_two': 0.591615714977313}
accuracy=0.8886621594429016
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3676205271315991, 'dropout_cnn_block_three': 0.3136528408107621, 'dropout_cnn_block_two': 0.32247195268648593, 'dropout_dense_block_one': 0.6527958568294003, 'dropout_dense_block_two': 0.3001247466782248}
accuracy=0.9612244963645935
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3102935380462264, 'dropout_cnn_block_three': 0.4150902602440875, 'dropout_cnn_block_two': 0.3567528983788038, 'dropout_dense_block_one': 0.5128022111472013, 'dropout_dense_block_two': 0.6819617160067641}
accuracy=0.9274376630783081
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3556508820473712, 'dropout_cnn_block_three': 0.3565698615765965, 'dropout_cnn_block_two': 0.46051345953095